# Языковые модели

Языковые модели играют важную роль в системах распознавания речи, помогая создавать более грамотные и лексически корректные тексты. В данной работе мы будем изучать нграмные языковые модели, которые позволяют довольно легко оценить вероятность и правдоподобность текста.

В нграмной языковой модели, нграм - это последовательность из n слов в тексте. Например, в предложении "по-моему мы сэкономим уйму времени если я сойду с ума прямо сейчас", биграмами будут "по-моему мы", "мы сэкономим", "сэкономим уйму" итд. Языковые модели оценивают вероятность появления последовательности слов, исходя из статистики появления каждого из нграм в обучающей выборке.

Порядком (order) нграм языковой модели называют максимальную длину нграм, которую учитывает модель. 

Практическая работа разделена на 2 части: 
1. Построение нграмой языковой модели - основная часть, 12 баллов
1. Предсказание с помощью языковой модели - дополнительная часть, 4 балла



Полезные сслыки:
* arpa формат - https://cmusphinx.github.io/wiki/arpaformat/
* обучающие материалы - https://pages.ucsd.edu/~rlevy/teaching/2015winter/lign165/lectures/lecture13/lecture13_ngrams_with_SRILM.pdf
* обучающие материалы.2 - https://cjlise.github.io/machine-learning/N-Gram-Language-Model/

In [42]:
import numpy as np
from collections import defaultdict
from typing import List, Dict, Tuple

# 1. Построение нграмной языковой модели. (12 баллов)


Вероятность текста с помощью нграмной языковой модели можно вычислить по формуле: 
$$ P(w_1, w_2, .., w_n) = {\prod{{P_{i=0}^{n}(w_i| w_{i-order}, .., w_{i-1})}}} $$

В простом виде, при обучении нграмной языковой модели, чтобы рассчитать условную вероятность каждой нграмы, используется формула, основанная на количестве появлений нграмы в обучающей выборке. Формула выглядит следующим образом:
$$ P(w_i| w_{i-order}, .., w_{i-1}) = {{count(w_{i-order}, .., w_{i})} \over {count(w_{i-order},..., w_{i-1})}} $$

Поскольку униграмы не содержат в себе какого-дибо контекста, вероятность униграмы можно посчитать поделив кол-во этой слова на общее количество слов в обучающей выборке. 


In [43]:
# в первую очередь нам понадобится подсчитать статистику по обучающей выборке 
def count_ngrams(train_text: List[str], order=3, bos=True, eos=True) -> Dict[Tuple[str], int]:
    ngrams = defaultdict(int)
    
    # TODO реализуйте функцию, которая подсчитывает все 1-gram 2-gram ... order-gram ngram'ы в тексте
    for phrase in train_text:
        words = phrase.strip().split(' ')
        if bos:
            words = ['<s>'] + words
        if eos:
            words = words + ['</s>']

        for ord in range(1, order+1):
            for start in range(len(words)-ord+1):
                gram = tuple(words[start:start+ord])
                if gram not in ngrams:
                    ngrams[gram] = 1
                else:
                    ngrams[gram] += 1
    # 
    return dict(ngrams)

In [44]:
count_ngrams(['привет привет как дела'], order=1, bos=True, eos=True)

{('<s>',): 1, ('привет',): 2, ('как',): 1, ('дела',): 1, ('</s>',): 1}

In [45]:
def test_count_ngrams():
    assert count_ngrams(['привет привет как дела'], order=1, bos=True, eos=True) == {
        ('<s>',): 1, 
        ('привет',): 2, 
        ('как',): 1, 
        ('дела',): 1, 
        ('</s>',): 1
    }
    assert count_ngrams(['привет привет как дела'], order=1, bos=False, eos=True) == {
        ('привет',): 2, 
        ('как',): 1, 
        ('дела',): 1, 
        ('</s>',): 1
    }
    assert count_ngrams(['привет привет как дела'], order=1, bos=False, eos=False) == {
        ('привет',): 2, 
        ('как',): 1, 
        ('дела',): 1
    }
    assert count_ngrams(['привет привет как дела'], order=2, bos=False, eos=False) == {
        ('привет',): 2, 
        ('как',): 1, 
        ('дела',): 1,
        ('привет', 'привет'): 1,
        ('привет', 'как'): 1,
        ('как', 'дела'): 1
    }    
    assert count_ngrams(['привет ' * 6], order=2, bos=False, eos=False) == {
        ('привет',): 6, 
        ('привет', 'привет'): 5
    }
    result = count_ngrams(['практическое сентября',
                           'второе практическое занятие пройдет в офлайне 32 сентября в 12 часов 32 минуты',
                           'в офлайне в 32 12'], order=5)
    assert result[('<s>',)] == 3
    assert result[('32',)] == 3
    assert result[('<s>', 'в', 'офлайне', 'в', '32')] == 1
    assert result[('офлайне', 'в', '32', '12', '</s>')] == 1
    print('Test 1a passed')
    
    
test_count_ngrams()  

Test 1a passed



Простой подход к вычислению вероятностей через количество нграм имеет существенный недостаток. Если в тексте встретится нграмма, которой не было в обучающей выборке, то вероятность всего текста будет равна нулю. 

Чтобы избежать данного недостатка, вводится специальное сглаживание - [сглаживание Лапласа](https://en.wikipedia.org/wiki/Additive_smoothing). Данная техника позволяет учитывать нграмы, не встретившиеся в обучающей выборке, и при этом не делает вероятность текста равной нулю.

Формула сглаживания Лапласа выглядит следующим образом:

$$ P(w_i| w_{i-order}, .., w_{i-1}) = {{count(w_{i-order}, .., w_{i}) + k} \over {count(w_{i-order},..., w_{i-1}) + k*V}} $$

Здесь V - количество слов в словаре, а k - гиперпараметр, который контролирует меру сглаживания. Как правило, значение k выбирается экспериментально, чтобы найти оптимальный баланс между учетом редких нграм и сохранением вероятности для часто встречающихся нграм.


In [46]:
count_ngrams(['практическое сентября',
                           'второе практическое занятие в офлайне 32 сентября в 12 часов 32 минуты',
                           'в офлайне в 32 12'], order=4)

{('<s>',): 3,
 ('практическое',): 2,
 ('сентября',): 2,
 ('</s>',): 3,
 ('<s>', 'практическое'): 1,
 ('практическое', 'сентября'): 1,
 ('сентября', '</s>'): 1,
 ('<s>', 'практическое', 'сентября'): 1,
 ('практическое', 'сентября', '</s>'): 1,
 ('<s>', 'практическое', 'сентября', '</s>'): 1,
 ('второе',): 1,
 ('занятие',): 1,
 ('в',): 4,
 ('офлайне',): 2,
 ('32',): 3,
 ('12',): 2,
 ('часов',): 1,
 ('минуты',): 1,
 ('<s>', 'второе'): 1,
 ('второе', 'практическое'): 1,
 ('практическое', 'занятие'): 1,
 ('занятие', 'в'): 1,
 ('в', 'офлайне'): 2,
 ('офлайне', '32'): 1,
 ('32', 'сентября'): 1,
 ('сентября', 'в'): 1,
 ('в', '12'): 1,
 ('12', 'часов'): 1,
 ('часов', '32'): 1,
 ('32', 'минуты'): 1,
 ('минуты', '</s>'): 1,
 ('<s>', 'второе', 'практическое'): 1,
 ('второе', 'практическое', 'занятие'): 1,
 ('практическое', 'занятие', 'в'): 1,
 ('занятие', 'в', 'офлайне'): 1,
 ('в', 'офлайне', '32'): 1,
 ('офлайне', '32', 'сентября'): 1,
 ('32', 'сентября', 'в'): 1,
 ('сентября', 'в', '12'): 1,
 ('

In [83]:
# функция подсчета вероятности через количество со сглаживанием Лапласа
def calculate_ngram_prob(ngram: Tuple[str], counts: Dict[Tuple[str], int], V=None, k=0) -> float:
    # подсчитывет ngram со сглаживанием Лапласа
    # V --- мощность словаря, количество уникальных слов в обучающей выборке
    V = 0
    for gram in counts:
        if len(gram) == 1: V += 1
    counts = defaultdict(int, counts)
    if ngram not in counts:
        num = k
    else:
        num = counts[ngram] + k

    if len(ngram) == 1:
        den = sum(value for key, value in counts.items() if len(key) == 1) + k*V
    else:
        den = counts[ngram[:-1]] + k * V

    prob = num/den

    return prob

In [86]:
def test_calculate_ngram_prob():
    counts = count_ngrams(['практическое сентября',
                           'второе практическое занятие в офлайне 32 сентября в 12 часов 32 минуты',
                           'в офлайне в 32 12'], order=4)
    assert calculate_ngram_prob(('в', 'офлайне'), counts) == 0.5
    assert calculate_ngram_prob(('в', ), counts) == 4/25
    assert calculate_ngram_prob(('в', ), counts, k=0.5) == (4+0.5)/(25+0.5*12)
    assert calculate_ngram_prob(('в', 'офлайне', 'в', '32'), counts) == 1.0
    assert calculate_ngram_prob(('в', 'офлайне'), counts, k=1) == 0.1875
    assert calculate_ngram_prob(('в', 'офлайне'), counts, k=0.5) == 0.25
    assert calculate_ngram_prob(('в', 'онлайне'), counts, k=0) == 0.0
    assert calculate_ngram_prob(('в', 'онлайне'), counts, k=1) == 0.0625
    assert calculate_ngram_prob(('в', 'офлайне'), counts, k=0.5) == 0.25

    print("Test 1.b passed")
    

test_calculate_ngram_prob()  

Test 1.b passed


Основной метрикой язковых моделей является перплексия. 

Перплексия  — безразмерная величина, мера того, насколько хорошо распределение вероятностей предсказывает выборку. Низкий показатель перплексии указывает на то, что распределение вероятности хорошо предсказывает выборку.

$$ ppl = {P(w_1, w_2 ,..., w_N)^{- {1} \over {N}}} $$


In [87]:
# Языковая модель 
class NgramLM:
    def __init__(self, order=3, bos=True, eos=True, k=1, predefined_vocab=None):
        self.order = order
        self.eos = eos
        self.bos = bos
        self.k = k
        self.vocab = predefined_vocab
        self.ngrams_count = None
        
    @property
    def V(self) -> int:
        return len(self.vocab)
    
    def fit(self, train_text: List[str]) -> None:
        # Подсчет vocab и ngrams_count по обучающей выборке
        self.ngrams_count = count_ngrams(train_text)
        self.vocab = [gram for gram in self.ngrams_count if len(gram) == 1]
    
    def predict_ngram_log_proba(self, ngram: Tuple[str]) -> float:
        # TODO 
        prob = np.log(calculate_ngram_prob(ngram, self.ngrams_count, k=self.k))
        return prob
        # считаем логарифм вероятности конкретной нграмы
           
    def predict_log_proba(self, words: List[str]) -> float:
        if self.bos:
            words = ['<s>'] + words
        if self.eos:
            words = words + ['</s>']
        logprob = 0
        # TODO 
        N = 0

        for tail in range(len(words)):
            start = tail - self.order + 1 if self.order < tail + 1 else 0
            logprob += self.predict_ngram_log_proba(tuple(words[start:tail+1]))
            
            N += 1

        # применяем chain rule, чтобы посчитать логарифм вероятности всей строки
        return logprob/N
        
    def ppl(self, text: List[str]) -> float:
        #TODO 
        # подсчет перплексии
        # Для того, чтобы ваш код был численно стабильным, 
        #    не считайте формулу напрямую, а воспользуйтесь переходом к логарифмам вероятностей
        #perplexity = 0
        
        probas = []
        
        for sentence in text:
            words = sentence.split()
            probas.append(self.predict_log_proba(words))    
    
        perplexity = np.exp(-np.mean(probas))
        
        return perplexity

In [88]:
def test_lm():
    train_data = ["по-моему мы сэкономим уйму времени если я сойду с ума прямо сейчас",
                  "если я сойду с ума прямо сейчас по-моему мы сэкономим уйму времени",
                  "мы сэкономим уйму времени если я сейчас сойду с ума по-моему"]
    global lm
    lm = NgramLM(order=2)
    lm.fit(train_data)
    assert lm.V == 14
    assert np.isclose(lm.predict_log_proba(['мы']), lm.predict_log_proba(["если"]))
    assert lm.predict_log_proba(["по-моему"]) > lm.predict_log_proba(["если"]) 
    
    gt = ((3+1)/(41 + 14) * 1/(3+14))**(-1/2)
    ppl = lm.ppl([''])
    assert  np.isclose(ppl, gt), f"{ppl=} {gt=}"
    
    gt = ((3+1)/(41 + 14) * 1/(3+14) * 1/(14)) ** (-1/3)
    ppl = lm.ppl(['ЧТО'])
    assert  np.isclose(ppl, gt), f"{ppl=} {gt=}"
    
    test_data = ["по-моему если я прямо сейчас сойду с ума мы сэкономим уйму времени"]
    ppl = lm.ppl(test_data)
    assert round(ppl, 2) == 7.33, f"{ppl}"
test_lm()

In [89]:
#https://paulapivat.medium.com/defaultdict-4e33767ee119

# 2. Предсказания с помощью языковой модели (4 балла)

In [220]:
def predict_next_word(lm: NgramLM, prefix: List[str], topk=4):
    # TODO реализуйте функцию, которая предсказывает продолжение фразы. 
    # верните topk наиболее вероятных продолжений фразы prefix 

    pred_scores = {}
        
    for word in lm.vocab:
        phrase = ' '. join(prefix + [word[0]]) #надо подать строку
        pred_scores[word] = lm.ppl([phrase])
    
    return sorted(pred_scores.items(), key=lambda x: x[1])[:topk]

In [221]:
train_data = ['привет привет как дела',
              'практическое сентября',
              'второе практическое занятие пройдет в офлайне тридцать второго сентября в двенадцать часов тридцать две минуты',
              'в офлайне в тридцать два двенадцать',
              'по-моему мы сэкономим уйму времени если я сойду с ума прямо сейчас',
              'если я сойду с ума прямо сейчас по-моему мы сэкономим уйму времени',
              'мы сэкономим уйму времени если я сейчас сойду с ума по-моему',
              'по-моему если я прямо сейчас сойду с ума мы сэкономим уйму времени']

Обучите языковую модель на всем тексте из этой лабораторной работы (не забудьте заранее нормализовать текст).

Что предскажет модель по префиксам `по-моему`, `сэкономим`, `нграм` и `вероятности`? 

In [228]:
lm = NgramLM(order = 2)
lm.fit(train_data)

In [234]:
#predict_next_word(lm, ['мы'])

In [229]:
predict_next_word(lm, ['по-моему'])

[(('мы',), 16.3446363250503),
 (('времени',), 16.3446363250503),
 (('дела',), 17.687585819239324),
 (('минуты',), 17.687585819239324)]

In [230]:
predict_next_word(lm, ['сэкономим'])

[(('уйму',), 18.93186452668453),
 (('времени',), 21.510751118558023),
 (('дела',), 23.278172048567892),
 (('минуты',), 23.278172048567892)]

In [231]:
predict_next_word(lm, ['нграм'])

[(('времени',), 20.867911577546664),
 (('дела',), 22.582513893591855),
 (('минуты',), 22.582513893591855),
 (('сентября',), 22.75690918992883)]

In [232]:
predict_next_word(lm, ['вероятности'])

[(('времени',), 20.867911577546664),
 (('дела',), 22.582513893591855),
 (('минуты',), 22.582513893591855),
 (('сентября',), 22.75690918992883)]